In [12]:
#import widget functionality
from __future__ import print_function
from ipywidgets import interact, interactive, fixed, interact_manual
from ipywidgets import FloatSlider
from ipywidgets import IntSlider
import ipywidgets as widgets

#Load libraries
import pandas as pd
import numpy as np
from IPython.display import display
import matplotlib.pyplot as plt
import math as math
import line_profiler
%load_ext line_profiler


The line_profiler extension is already loaded. To reload it, use:
  %reload_ext line_profiler


In [ ]:
#Creates array to hold the frequency of each barcode in the library
def my_data(barcode_amount, broadness, exponentiality):
    data = np.zeros((barcode_amount, 2))

    num = 1
    for row in range(barcode_amount):
        data[row][0] = num
        num += 1

    for row in range(barcode_amount):
        data[row][1] = math.exp(exponentiality/broadness*(data[row][0]))
    
    return data

#Calculates the probability of picking each barcode
def probability_distribution(barcode_distribution): 
    barcode_probability = np.zeros(len(barcode_distribution))
    counter = 0
    for barcode in barcode_distribution[:,1]:
        probability = barcode_distribution[counter,1] / sum(barcode_distribution[:,1])
        barcode_probability[counter] = probability
        counter += 1
    
    return barcode_probability

#Calculates the fraction of uniquely labelled cells, given the barcode probability distribution and number of infected cells
def fraction_unique(barcode_probability, cell_num):
    barcode_contribution = barcode_probability * (1 - (1 - barcode_probability)**(cell_num - 1))
    unique = 1 - sum(barcode_contribution)
    
    return unique

In [31]:
#Defines barcode distribution with editable parameters, outputting visualisation
def generate_data(barcode_amount, broadness, exponentiality):
    plt.figure(figsize=(15,7))
    plt.plot(my_data(barcode_amount, broadness, exponentiality)[:,0], 
        my_data(barcode_amount, broadness, exponentiality)[:,1])
    plt.title('Barcode diversity')
    plt.xlabel('Barcode index')
    plt.ylabel('Barcode frequency')
    plt.xlim(0)
    plt.grid(True)
    global barcode_distribution
    barcode_distribution = my_data(barcode_amount, broadness, exponentiality)

interact(generate_data, barcode_amount=IntSlider(min = 100, max = 10000, value = 5000, continuous_update=False), 
                        broadness=IntSlider(min = 1, max = 1000, value = 200, continuous_update=False), 
                        exponentiality=FloatSlider(min = 0, max = 3, step = 1e-2, value = 1, continuous_update=False))

#Takes about 1 sec per thousand barcodes + 1 sec per thousand cells
def plot_unique_barcoded_cells(max_cells, plotted_points = (1,10000,10)):
    plt.figure(figsize=(15,7))
    cell_num_array = np.arange(1, plotted_points, dtype = float)
    print(cell_num_array)
    barcode_probability = probability_distribution(barcode_distribution)
    for num in cell_num_array:
        cell_num_array[int(num)-1] = fraction_unique(barcode_probability, int(num))
    plt.plot(np.arange(1, max_cells, max_cells//plotted_points), cell_num_array)
    plt.title('Barcode diversity')
    plt.xlabel('Number of infected starter cells')
    plt.ylabel('Fraction of uniquely labeled cells')
    plt.xlim(0)
    plt.ylim(0, 1)
    plt.grid(True)

interact_manual(plot_unique_barcoded_cells, max_cells=IntSlider(min = 1, max = 10000, value = 5000), 
                                            plotted_points=IntSlider(min = 1, max = 10000, value = 5000))

interactive(children=(IntSlider(value=5000, continuous_update=False, description='barcode_amount', max=10000, …

interactive(children=(IntSlider(value=5000, description='max_cells', max=10000, min=1), IntSlider(value=5000, …

<function __main__.plot_unique_barcoded_cells(max_cells, plotted_points=(1, 10000, 10))>

In [ ]:
from barcode_functions import plot_unique_barcoded_cells
%lprun -f plot_unique_barcoded_cells plot_unique_barcoded_cells()